In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("CourseEnrollments").getOrCreate()

Data Loading

1. Load the data with schema inference enabled.

In [0]:
df_infer = spark.read.option("header", "true").option("inferSchema", "true").csv("file:/Workspace/Shared/course_enrollments.csv")
df_infer.show()
df_infer.printSchema()

+------------+-----------+--------------------+-----------+----------+---------------+------+---------+
|EnrollmentID|StudentName|          CourseName|   Category|EnrollDate|ProgressPercent|Rating|   Status|
+------------+-----------+--------------------+-----------+----------+---------------+------+---------+
|      ENR001|     Aditya|Python for Beginners|Programming|2024-05-10|             80|   4.5|   Active|
|      ENR002|     Simran|Data Analysis wit...|  Analytics|2024-05-12|            100|   4.7|Completed|
|      ENR003|     Aakash| Power BI Essentials|  Analytics|2024-05-13|             30|   3.8|   Active|
|      ENR004|       Neha|         Java Basics|Programming|2024-05-15|              0|  NULL| Inactive|
|      ENR005|       Zara|Machine Learning 101|         AI|2024-05-17|             60|   4.2|   Active|
|      ENR006|    Ibrahim|Python for Beginners|Programming|2024-05-18|             90|   4.6|Completed|
+------------+-----------+--------------------+-----------+-----

2. Manually define schema and compare both approaches.

In [0]:
schema = StructType([
    StructField("EnrollmentID", StringType(), True),
    StructField("StudentName", StringType(), True),
    StructField("CourseName", StringType(), True),
    StructField("Category", StringType(), True),
    StructField("EnrollDate", DateType(), True),
    StructField("ProgressPercent", IntegerType(), True),
    StructField("Rating", DoubleType(), True),
    StructField("Status", StringType(), True)
])

df_manual = spark.read.option("header", "true").option("inferSchema", "true").csv("file:/Workspace/Shared/course_enrollments.csv")
df_manual.printSchema()


root
 |-- EnrollmentID: string (nullable = true)
 |-- StudentName: string (nullable = true)
 |-- CourseName: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- EnrollDate: date (nullable = true)
 |-- ProgressPercent: integer (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Status: string (nullable = true)



Filtering and Transformation

3. Filter records where ProgressPercent < 50 .

In [0]:
df_filtered = df_manual.filter(col("ProgressPercent") < 50)
df_filtered.show()

+------------+-----------+-------------------+-----------+----------+---------------+------+--------+
|EnrollmentID|StudentName|         CourseName|   Category|EnrollDate|ProgressPercent|Rating|  Status|
+------------+-----------+-------------------+-----------+----------+---------------+------+--------+
|      ENR003|     Aakash|Power BI Essentials|  Analytics|2024-05-13|             30|   3.8|  Active|
|      ENR004|       Neha|        Java Basics|Programming|2024-05-15|              0|  NULL|Inactive|
+------------+-----------+-------------------+-----------+----------+---------------+------+--------+



4. Replace null ratings with average rating.

In [0]:
avg_rating = df_manual.select(avg("Rating")).first()[0]
df_filled = df_manual.fillna({"Rating": avg_rating})

5. Add column IsActive → 1 if Status is Active, else 0.

In [0]:
df_flagged = df_filled.withColumn("IsActive", when(col("Status") == "Active", 1).otherwise(0))

Aggregations & Metrics

6. Find average progress by course.

In [0]:
df_flagged.groupBy("CourseName").agg(avg("ProgressPercent").alias("AvgProgress")).show()

+--------------------+-----------+
|          CourseName|AvgProgress|
+--------------------+-----------+
|Data Analysis wit...|      100.0|
|         Java Basics|        0.0|
|Machine Learning 101|       60.0|
|Python for Beginners|       85.0|
| Power BI Essentials|       30.0|
+--------------------+-----------+



7. Get count of students in each course category.

In [0]:
df_flagged.groupBy("Category").count().show()

+-----------+-----+
|   Category|count|
+-----------+-----+
|Programming|    3|
|         AI|    1|
|  Analytics|    2|
+-----------+-----+



8. Identify the most enrolled course.

In [0]:
df_flagged.groupBy("CourseName").count().orderBy(desc("count")).limit(1).show()

+--------------------+-----+
|          CourseName|count|
+--------------------+-----+
|Python for Beginners|    2|
+--------------------+-----+



Joins

10. Join course_enrollments with course_details to include duration and
instructor.

In [0]:
df_details =  spark.read.option("header", "true").option("inferSchema", "true").csv("file:/Workspace/Shared/course_details.csv")
df_joined = df_flagged.join(df_details, on="CourseName", how="left")
df_joined.show()


+--------------------+------------+-----------+-----------+----------+---------------+-----------------+---------+--------+-------------+----------+
|          CourseName|EnrollmentID|StudentName|   Category|EnrollDate|ProgressPercent|           Rating|   Status|IsActive|DurationWeeks|Instructor|
+--------------------+------------+-----------+-----------+----------+---------------+-----------------+---------+--------+-------------+----------+
|Python for Beginners|      ENR001|     Aditya|Programming|2024-05-10|             80|              4.5|   Active|       1|            4|    Rakesh|
|Data Analysis wit...|      ENR002|     Simran|  Analytics|2024-05-12|            100|              4.7|Completed|       0|            3|    Anjali|
| Power BI Essentials|      ENR003|     Aakash|  Analytics|2024-05-13|             30|              3.8|   Active|       1|            5|     Rekha|
|         Java Basics|      ENR004|       Neha|Programming|2024-05-15|              0|4.359999999999999| I

Window Functions

11. Rank students in each course based on ProgressPercent .

In [0]:
window_rank = Window.partitionBy("CourseName").orderBy(desc("ProgressPercent"))
df_ranked = df_joined.withColumn("Rank", rank().over(window_rank))
df_ranked.select("StudentName", "CourseName", "ProgressPercent", "Rank").show()

+-----------+--------------------+---------------+----+
|StudentName|          CourseName|ProgressPercent|Rank|
+-----------+--------------------+---------------+----+
|     Simran|Data Analysis wit...|            100|   1|
|       Neha|         Java Basics|              0|   1|
|       Zara|Machine Learning 101|             60|   1|
|     Aakash| Power BI Essentials|             30|   1|
|    Ibrahim|Python for Beginners|             90|   1|
|     Aditya|Python for Beginners|             80|   2|
+-----------+--------------------+---------------+----+



12. Get lead and lag of EnrollDate by Category.

In [0]:
window_leadlag = Window.partitionBy("Category").orderBy("EnrollDate")
df_laglead = df_joined.withColumn("PrevDate", lag("EnrollDate").over(window_leadlag)) \
                      .withColumn("NextDate", lead("EnrollDate").over(window_leadlag))
df_laglead.select("StudentName", "Category", "EnrollDate", "PrevDate", "NextDate").show()

+-----------+-----------+----------+----------+----------+
|StudentName|   Category|EnrollDate|  PrevDate|  NextDate|
+-----------+-----------+----------+----------+----------+
|       Zara|         AI|2024-05-17|      NULL|      NULL|
|     Simran|  Analytics|2024-05-12|      NULL|2024-05-13|
|     Aakash|  Analytics|2024-05-13|2024-05-12|      NULL|
|     Aditya|Programming|2024-05-10|      NULL|2024-05-15|
|       Neha|Programming|2024-05-15|2024-05-10|2024-05-18|
|    Ibrahim|Programming|2024-05-18|2024-05-15|      NULL|
+-----------+-----------+----------+----------+----------+



Pivoting & Formatting

13. Pivot data to show total enrollments by Category and Status.

In [0]:
df_joined.groupBy("Category").pivot("Status").count().show()

+-----------+------+---------+--------+
|   Category|Active|Completed|Inactive|
+-----------+------+---------+--------+
|Programming|     1|        1|       1|
|         AI|     1|     NULL|    NULL|
|  Analytics|     1|        1|    NULL|
+-----------+------+---------+--------+



14. Extract year and month from EnrollDate .

In [0]:
df_dated = df_joined.withColumn("EnrollYear", year("EnrollDate")) \
                    .withColumn("EnrollMonth", month("EnrollDate"))

Cleaning and Deduplication

15. Drop rows where Status is null or empty.

In [0]:
df_cleaned = df_dated.filter(col("Status").isNotNull() & (col("Status") != ""))
df_cleaned.show()

+--------------------+------------+-----------+-----------+----------+---------------+-----------------+---------+--------+-------------+----------+----------+-----------+
|          CourseName|EnrollmentID|StudentName|   Category|EnrollDate|ProgressPercent|           Rating|   Status|IsActive|DurationWeeks|Instructor|EnrollYear|EnrollMonth|
+--------------------+------------+-----------+-----------+----------+---------------+-----------------+---------+--------+-------------+----------+----------+-----------+
|Python for Beginners|      ENR001|     Aditya|Programming|2024-05-10|             80|              4.5|   Active|       1|            4|    Rakesh|      2024|          5|
|Data Analysis wit...|      ENR002|     Simran|  Analytics|2024-05-12|            100|              4.7|Completed|       0|            3|    Anjali|      2024|          5|
| Power BI Essentials|      ENR003|     Aakash|  Analytics|2024-05-13|             30|              3.8|   Active|       1|            5|   

16. Remove duplicate enrollments using dropDuplicates() .

In [0]:
df_deduped = df_cleaned.dropDuplicates(["EnrollmentID"])
df_deduped.show()

+--------------------+------------+-----------+-----------+----------+---------------+-----------------+---------+--------+-------------+----------+----------+-----------+
|          CourseName|EnrollmentID|StudentName|   Category|EnrollDate|ProgressPercent|           Rating|   Status|IsActive|DurationWeeks|Instructor|EnrollYear|EnrollMonth|
+--------------------+------------+-----------+-----------+----------+---------------+-----------------+---------+--------+-------------+----------+----------+-----------+
|         Java Basics|      ENR004|       Neha|Programming|2024-05-15|              0|4.359999999999999| Inactive|       0|            6|     Manoj|      2024|          5|
|Machine Learning 101|      ENR005|       Zara|         AI|2024-05-17|             60|              4.2|   Active|       1|            8|     Samir|      2024|          5|
|Data Analysis wit...|      ENR002|     Simran|  Analytics|2024-05-12|            100|              4.7|Completed|       0|            3|   

Export

Write the final cleaned DataFrame to:
CSV (overwrite mode)

In [0]:
df_deduped.write.mode("overwrite").csv("output/final_enrollments_csv", header=True)

JSON (overwrite mode)

In [0]:
df_deduped.write.mode("overwrite").json("output/final_enrollments_json")

Parquet (snappy compression)

In [0]:
df_deduped.write.mode("overwrite").option("compression", "snappy").parquet("output/final_enrollments_parquet")